## Colab

In [57]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installation

In [58]:
!pip install -qU langchain-community faiss-cpu langchain-openai langchain langchainhub tiktoken

## Path to FAISS index

In [71]:
import os
from langchain_openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = userdata.get('openAI')
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Index Path(Directory to save initialized index)
DRIVE_PATH = '/content/drive/MyDrive/RAG_JSON_EMBEDDINGS_INDEX'
INDEX_DIR_PATH = os.path.join(DRIVE_PATH, "INDEX")
HA_INDEX_PATH = os.path.join(INDEX_DIR_PATH, "HA_TEST_INDEX")
if not os.path.exists(INDEX_DIR_PATH):
    os.makedirs(INDEX_DIR_PATH)

if not os.path.exists(HA_INDEX_PATH):
    os.makedirs(HA_INDEX_PATH)

faiss_index_path = os.path.join(HA_INDEX_PATH, "test_faiss_index")

## Applying Exponenial Backoff

In [72]:
# Faiss index set
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

loaded_vector_store = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

In [73]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """
    You are an expert planning a date for loved one, family and friends.
    Your task is retrieving relevant data to generate a date plan.
    You have access to a database of locations for dating in Seoul.
    Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector
    database.
    By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.


    Each row in the table represents a location and its featrues.
    Features are separated by [SEP].
    If a row have 'None' in the feature, it means that the row doens't have that feature.
    Every row is in Korean while column names are in English.
    Provide these alternative questions separated by newlines.
    Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [74]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

retriever = loaded_vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 30}
)

# Retrieval chain
retrieval_chain = generate_queries | retriever.map() | get_unique_union

In [75]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
question = "성북구에서 할 수 있는 식도락 데이트!"
# RAG
template = """
- You are a helpful assistant that answers questions about the context below.
- You do not make up answers to questions that cannot be found in the context.
- If you don't know the answer to a question, just say that you don't know. Don't try to make up an answer.
- You will generate a list of activities and please follow the format:
[
  {{
    "activityTitle": Get the name of the place,
    "activityLoc": Get the address of the place,
    "timeTotal": Generate your expected time about the place or just put 1 hour,
    "activityDescription": Generate a description of the place based on your understanding,
    "activityImage": Get url of the place
  }},
  ...
]
- Make sure the list contains at least 5 activities
- You have to answer in Korean.

Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatOpenAI(temperature=0).with_structured_output(Search)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

In [83]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def completion_with_backoff(qa_chain, question):
    return qa_chain.invoke({"question":question})

In [80]:
print(final_rag_chain.invoke({"question": "성북구에서 할 수 있는 식도락 데이트"}))

[
  {
    "activityTitle": "천상애장어가",
    "activityLoc": "서울 성북구 동소문로6길 22 1층",
    "timeTotal": "1시간",
    "activityDescription": "신선한 장어와 다양한 반찬을 제공하는 장어 전문점으로, 편안한 분위기에서 맛있는 식사를 즐길 수 있습니다. 사장님의 친절한 서비스와 함께하는 소중한 시간을 보낼 수 있는 곳입니다.",
    "activityImage": "https://naver.me/FYuVRZ09"
  },
  {
    "activityTitle": "국민낙곱새 길음점",
    "activityLoc": "서울 성북구 삼양로 35 1층 102호",
    "timeTotal": "1시간",
    "activityDescription": "성북구의 대표 낙곱새 맛집으로, 위생적인 환경에서 맛있는 낙곱새를 즐길 수 있습니다. 친절한 서비스와 함께하는 맛있는 식사가 가능합니다.",
    "activityImage": "https://naver.me/5X9gjgLf"
  },
  {
    "activityTitle": "거기식당",
    "activityLoc": "서울 성북구 정릉로9길 14-5",
    "timeTotal": "1시간",
    "activityDescription": "국민대 후문 근처에 위치한 한식당으로, 정갈한 음식과 친절한 서비스가 특징입니다. 오랜만에 방문해도 반가운 추억을 떠올리게 하는 곳입니다.",
    "activityImage": "https://naver.me/xpPLYNIO"
  },
  {
    "activityTitle": "이향 본점",
    "activityLoc": "서울 성북구 성북로26길 13",
    "timeTotal": "1시간",
    "activityDescription": "깔끔한 한정식과 정성 가득한 약선밥을 제공하는 맛집으로, 건강한 식사를 원하시는 분들에게 추천합니다. 조용한 

In [85]:
print(completion_with_backoff(final_rag_chain, "성북구에서 할 수 있는 식도락 데이트!"))

[
  {
    "activityTitle": "천상애장어가",
    "activityLoc": "서울 성북구 동소문로6길 22 1층",
    "timeTotal": "1시간",
    "activityDescription": "신선한 장어와 다양한 반찬, 국수까지 맛볼 수 있는 장어 전문점입니다. 사장님의 친절한 서비스와 아늑한 분위기로 데이트에 적합한 장소입니다.",
    "activityImage": "https://naver.me/FYuVRZ09"
  },
  {
    "activityTitle": "누브티스 성북동점",
    "activityLoc": "서울 성북구 선잠로 42",
    "timeTotal": "1시간",
    "activityDescription": "아름다운 정원과 독특한 인테리어가 매력적인 이탈리안 퓨전 레스토랑입니다. 파스타와 피자가 특히 맛있어 데이트에 적합한 장소입니다.",
    "activityImage": "https://naver.me/xw694Mb0"
  },
  {
    "activityTitle": "거기식당",
    "activityLoc": "서울 성북구 정릉로9길 14-5",
    "timeTotal": "1시간",
    "activityDescription": "국민대 후문에 위치한 한식당으로, 정갈한 반찬과 맛있는 한식을 제공합니다. 오랜만에 방문해도 변함없는 맛이 매력적입니다.",
    "activityImage": "https://naver.me/xpPLYNIO"
  },
  {
    "activityTitle": "이향 본점",
    "activityLoc": "서울 성북구 성북로26길 13",
    "timeTotal": "1시간",
    "activityDescription": "건강한 나물 돌솥밥을 제공하는 한정식집으로, 깔끔한 반찬과 정성 가득한 약선밥이 특징입니다. 조용한 분위기에서 식사하기 좋습니다.",
    "activityImage": "https://

## Testing Trial

In [ ]:
import time
from time import sleep

for i in range(100):
  start_time = time.time()
  final_rag_chain.invoke({"question":"성북구에서 할 수 있는 식도락 데이트!"})
  end_time = time.time()
  print(f"{i+1} Elapsed Time: {end_time - start_time:.2f} seconds")

for i in range(100):
  start_time = time.time()
  completion_with_backoff(final_rag_chain, question = "성북구에서 할 수 있는 식도락 데이트!")
  end_time = time.time()
  print(f"{i+1} Elapsed Time: {end_time - start_time:.2f} seconds")